In [1]:
# https://github.com/Grillard/GalfitPyWrap may be useful for setting up inputs
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import glob
from scipy import ndimage
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.io import ascii
from astropy import wcs
from astropy.table import Table, hstack, join
from __future__ import division
import fnmatch
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0,'/data/emiln/XLSSC122_GalPops/Analysis/Modules')
import galfit_helpers as gfh

/data/emiln/miniconda2/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


### Load catalog

In [4]:
HST_cat_root = '/data/emiln/XLSSC122_GalPops/Data/HST/Products/catalogs/'
HST_cat_filename = HST_cat_root+'xlssc122_F105_F140_redshifts_short_v4_snr_em_member2_by_hand.cat'
full_df, member_df = gfh.load_HST_galaxy_catalog(HST_cat_filename)

### Load images

In [10]:
hst_file = '/data/emiln/XLSSC122_GalPops/Data/HST/Raw/xlssuj0217-0345-f140w_drz_sci.fits'
hst_wht_file = '/data/emiln/XLSSC122_GalPops/Data/HST/Raw/xlssuj0217-0345-f140w_drz_wht.fits'

hst_hdulist = fits.open(hst_file)
full_wcs = wcs.WCS(hst_hdulist[0].header)
wht_hdulist = fits.open(hst_wht_file)
# exp_hdulist = fits.open(hst_exp_file) # No exposure map for our data


### Make cutouts

In [13]:
# For each object in fhst_df
# Make a directory to store data products
# Create image cutout with same header and updated wcs info
# Create cutout sigma image in units of cps

import copy

w=400
hstexptime=5170 #seconds

df = member_df

for idx, r in df.iterrows():
    ra = r.ra
    dec = r.dec
    pixcrd = full_wcs.wcs_world2pix(ra,dec, 1)
    print pixcrd
    X = int(pixcrd[0])
    Y = int(pixcrd[1])
    ID = int(r['ID'])
    
    print "ID", ID
    print "RA:",ra
    print "DEC:",dec
    print "Initial X:", X
    print "Initial Y:", Y
    
    tdir = '/data/emiln/XLSSC122_GalPops/Data/HST/Products/galfit_results/'+str(ID)
    try:
        os.mkdir(tdir)
    except:
        print("Directory already exists.")

    wmapcut = full_wcs[Y-w:Y+w,X-w:X+w]
    datacut = hst_hdulist[0].data[Y-w:Y+w,X-w:X+w]
    # expcut = exp_hdulist[0].data[Y-w:Y+w,X-w:X+w]
#     rmscut = rms_hdulist[0].data[Y-w:Y+w,X-w:X+w]
    whtcut = wht_hdulist[0].data[Y-w:Y+w,X-w:X+w] # inverse variance
    rmscut = np.sqrt(1./whtcut) # stdev per pixel
    # print expcut.shape
    print datacut.shape
    print rmscut.shape
    print whtcut.shape

    newdata = datacut
    newwht = whtcut
    newcounts = newdata * hstexptime 
    
    nonzero_counts = newcounts.copy()
    nonzero_counts[nonzero_counts<0] = 0

    
    # RMS + POISSON
    newsigma_rms = rmscut + np.sqrt(nonzero_counts)/hstexptime

    new_cos_hdu = fits.PrimaryHDU(newdata)
    new_cos_hdul = fits.HDUList([new_cos_hdu])
    new_cos_hdul[0].header = hst_hdulist[0].header
    new_cos_hdul[0].header.update(wmapcut.to_header()) # All centers will be at (100,100) in physical/image coords after this
    new_cos_hdul[0].header['EXPTIME'] = 1 #CPS
    new_cos_hdul[0].header['GAIN'] = 2.5 #CPS
    new_cos_filename = tdir+'/data_cps.fits'
    new_cos_hdul.writeto(new_cos_filename, clobber=True)

    # new_sigma_hdu = fits.PrimaryHDU(newsigma_rms_meanexp)
    new_sigma_hdu = fits.PrimaryHDU(newsigma_rms)
    new_sigma_hdul = fits.HDUList([new_sigma_hdu])
    new_sigma_hdul[0].header = copy.deepcopy(wht_hdulist[0].header)
    new_sigma_hdul[0].header.update(wmapcut.to_header()) # All centers will be at (100,100) in physical/image coords after this
    new_sigma_hdul[0].header['EXPTIME'] = 1 #CPS
    new_cos_hdul[0].header['GAIN'] = 2.5 #CPS
    # new_sigma_filename = tdir+'/sigma_rms_meanexp_cps.fits'
    new_sigma_filename = tdir+'/sigma_rms_cps.fits'
    new_sigma_hdul.writeto(new_sigma_filename, clobber=True)

[array(2197.56319443), array(2196.52377215)]
ID 529
RA: 34.434215
DEC: -3.758796
Initial X: 2197
Initial Y: 2196
(800, 800)
(800, 800)
(800, 800)
[array(2911.99895553), array(1913.49833046)]
ID 455
RA: 34.422282
DEC: -3.7635129999999997
Initial X: 2911
Initial Y: 1913
(800, 800)
(800, 800)
(800, 800)
[array(2204.68789793), array(2264.80376615)]
ID 661
RA: 34.434096000000004
DEC: -3.757658
Initial X: 2204
Initial Y: 2264
(800, 800)
(800, 800)
(800, 800)
[array(2303.23688037), array(2728.84358876)]
ID 1036
RA: 34.43245
DEC: -3.7499239999999996
Initial X: 2303
Initial Y: 2728
(800, 800)
(800, 800)
(800, 800)
[array(2148.70859814), array(1647.40378369)]
ID 300
RA: 34.435031
DEC: -3.7679480000000005
Initial X: 2148
Initial Y: 1647
(800, 800)
(800, 800)
(800, 800)
[array(2112.96528327), array(2535.70377129)]
ID 920
RA: 34.435628
DEC: -3.753143
Initial X: 2112
Initial Y: 2535
(800, 800)
(800, 800)
(800, 800)
[array(1422.9608133), array(1643.85862651)]
ID 305
RA: 34.447153
DEC: -3.768007
Initi